In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

In [2]:
df_train = pd.read_csv('new_train_data.tsv',na_values=['?',' '])
df_train

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body,url_boilerplate
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,0.0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...,bloomberg news 2010 12 23 ibm predicts hologra...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,0.0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...,popsci technology article 2012 07 electronic f...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,0.0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...,menshealth health flu fighting fruits cm mmc F...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0.0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...,dumblittleman 2007 12 10 foolproof tips for be...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,0.0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...,bleacherreport articles 1205138 the 50 coolest...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,http://techcrunch.com/2010/09/08/kno-raises-46...,8958,"{""title"":""Kno Raises 46 Million More To Build ...",computer_internet,0.651067,3.010526,0.474747,0.222222,0.191919,0.191919,...,0.0,2219,99,11,0.040404,0.071429,0,Kno Raises 46 Million More To Build Most Power...,Marc Andreessen is normally enthusiastic about...,techcrunch 2010 09 08 kno raises 46 million mo...
7391,http://www.uncoached.com/category/why-i-miss-c...,8895,"{""title"":""Why I Miss College "",""body"":""Mar 30 ...",culture_politics,0.141920,2.208054,0.483333,0.246667,0.036667,0.026667,...,0.0,5672,300,4,0.020000,0.109453,0,Why I Miss College,Mar 30 2009 I d like to congratulate Jane on h...,uncoached category why i miss college
7392,http://eatthis.menshealth.com/slide/sweet-pota...,1191,"{""title"":""Sweet Potatoes Eat This Not That i'...",recreation,0.196273,2.000000,0.315789,0.171053,0.105263,0.052632,...,0.0,848,76,5,0.434211,0.117647,1,Sweet Potatoes Eat This Not That i'm eating t...,They re loaded with vitamin C which smoothes o...,eatthis menshealth slide sweet potatoes slides...
7393,http://naturallyella.com/,5612,"{""title"":""Naturally Ella "",""body"":"" "",""url"":""n...",arts_entertainment,0.617876,1.026316,0.210526,0.052632,0.000000,0.000000,...,1.0,386,38,0,0.026316,0.333333,1,Naturally Ella,NaN,naturallyella


In [3]:
X, Y = df_train['boilerplate'],df_train['label'] 
X, Y

(0       {"title":"IBM Sees Holographic Calls Air Breat...
 1       {"title":"The Fully Electronic Futuristic Star...
 2       {"title":"Fruits that Fight the Flu fruits tha...
 3       {"title":"10 Foolproof Tips for Better Sleep "...
 4       {"title":"The 50 Coolest Jerseys You Didn t Kn...
                               ...                        
 7390    {"title":"Kno Raises 46 Million More To Build ...
 7391    {"title":"Why I Miss College ","body":"Mar 30 ...
 7392    {"title":"Sweet Potatoes Eat This Not That  i'...
 7393    {"title":"Naturally Ella ","body":" ","url":"n...
 7394    {"title":"Esti Ginzburg Swimsuit by Letarte by...
 Name: boilerplate, Length: 7395, dtype: object,
 0       0
 1       1
 2       1
 3       1
 4       0
        ..
 7390    0
 7391    0
 7392    1
 7393    1
 7394    0
 Name: label, Length: 7395, dtype: int64)

In [4]:
Y.value_counts()

1    3796
0    3599
Name: label, dtype: int64

In [5]:
X.fillna("", inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=102)

In [6]:
stop_words = stopwords.words('english')
stop_word_add = ['becau','abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv']
for i in stop_word_add:
    stop_words.append(i)
    
def preprocessor(text):
    text = re.sub('[\W0-9]+', ' ', text.lower())
    return text

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# Train với split ,không dùng tập test

In [39]:
tfidf = TfidfVectorizer(stop_words = stop_words,
                        tokenizer = tokenizer_porter,
                        preprocessor = preprocessor)

clf = Pipeline([('vect', tfidf),
                ('pca',TruncatedSVD(n_components=300)),
                ('clf', VotingClassifier(estimators=[
                    ('clf1',LogisticRegression()),
                    ('clf2',LGBMClassifier()),
                    ('clf3',SVC(probability=True)),
                    ('clf4',GradientBoostingClassifier()),
                    ('clf5',RandomForestClassifier())], voting='soft'))])
clf.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocessor at 0x7f93280847b8>,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function tokenizer_porter at 0x7f9327674ea0>)),
                ('pca', TruncatedSVD(n_components=300)),
                ('clf',
                 VotingClassifier(estimators=[('clf1', LogisticRegression()),
               

In [40]:
y_pred = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred)

0.8676752955117326

# Train với full tập train, không sử dụng tập test 

In [11]:
clf.fit(X,Y)

Pipeline(steps=[('vect',
                 TfidfVectorizer(preprocessor=<function preprocessor at 0x7f93581c77b8>,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function tokenizer_porter at 0x7f9329a1aa60>)),
                ('pca', TruncatedSVD(n_components=300)),
                ('clf',
                 VotingClassifier(estimators=[('clf1', LogisticRegression()),
               

In [7]:
# read tập test 
df_test = pd.read_csv('new_test_data.tsv',na_values=['?',' '])
df_test

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,title,body,url_boilerplate
0,http://www.lynnskitchenadventures.com/2009/04/...,5865,"{""title"":""Homemade Enchilada Sauce Lynn s Kitc...",recreation,0.443906,2.558140,0.389706,0.257353,0.044118,0.022059,...,15,0.0,5643,136,3,0.242647,0.080597,Homemade Enchilada Sauce Lynn s Kitchen Advent...,I usually buy my enchilada sauce Yes I knew I ...,lynnskitchenadventures 2009 04 homemade enchil...
1,http://lolpics.se/18552-stun-grenade-ar,782,"{""title"":""lolpics Stun grenade ar "",""body"":"" f...",culture_politics,0.135844,3.771429,0.461538,0.205128,0.051282,0.000000,...,62,0.0,382,39,2,0.128205,0.176471,lolpics Stun grenade ar,funny pictures at lolpics.se. the best funny ...,lolpics se 18552 stun grenade ar
2,http://www.xcelerationfitness.com/treadmills.html,6962,"{""title"":""Treadmills "",""body"":"" treadmills, st...",NaN,NaN,2.269565,0.495726,0.384615,0.170940,0.170940,...,42,0.0,2420,117,1,0.581197,0.125000,Treadmills,"treadmills, stair, climbers, treadmills",xcelerationfitness treadmills html
3,http://www.bloomberg.com/news/2012-02-06/syria...,7640,"{""title"":""Father s Tactics Used by Assad to Cr...",culture_politics,0.902590,2.523490,0.705502,0.346278,0.122977,0.090615,...,41,0.0,5559,309,10,0.038835,0.063126,Father s Tactics Used by Assad to Crush Revolt...,Enlarge image Syrian President Bashar al Assad...,bloomberg news 2012 02 06 syria s assad deploy...
4,http://www.wired.com/gadgetlab/2011/12/stem-tu...,3589,"{""title"":""Stem Turns Lemons and Limes Into Jui...",science_technology,0.486363,1.848000,0.470968,0.161290,0.032258,0.000000,...,34,0.0,2209,155,10,0.096774,0.065341,Stem Turns Lemons and Limes Into Juicy Atomize...,Quirky s Stem turns any citrus fruit into an a...,wired gadgetlab 2011 12 stem turns lemons and ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166,http://busy-mommy.com/2012/02/peep-brownie-smo...,7264,"{""title"":""Peep Brownie S mores Busy Mommy An I...",NaN,NaN,1.666667,0.376623,0.129870,0.116883,0.090909,...,16,0.0,2772,77,3,0.012987,0.063401,Peep Brownie S mores Busy Mommy An Iowa Mom Blog,Easter candy is hitting the shelves and I ve b...,busy mommy 2012 02 peep brownie smores html
3167,http://www.cannabissearch.com/edibles/cheesecake/,9714,"{""url"":""cannabissearch edibles cheesecake"",""ti...",NaN,NaN,1.305556,0.654321,0.123457,0.024691,0.000000,...,6,0.0,6058,81,2,0.333333,0.061995,Marijuana infused cheesecake Medical Cannabis ...,Cannabis infused cheesecake is one of the tast...,cannabissearch edibles cheesecake
3168,http://www.tastespotting.com/popular/views/all...,5903,"{""title"":""Most Viewed Submissions All Time mos...",NaN,NaN,0.717277,0.291667,0.182292,0.000000,0.000000,...,19,0.0,2876,192,4,0.177083,0.117647,Most Viewed Submissions All Time most viewed s...,www tastespotting com January 27 is National C...,tastespotting popular views all time 2
3169,http://lifehacker.com/5839197/how-to-get-a-ful...,3176,"{""title"":""How to Get a Complete Workout with N...",sports,0.424304,0.940000,0.183333,0.066667,0.016667,0.016667,...,3,0.0,21029,180,12,0.333333,0.111966,How to Get a Complete Workout with Nothing But...,Everyone knows exercise plays an important rol...,lifehacker 5839197 how to get a full body work...


In [8]:
X_full_test = df_test['boilerplate']
X_full_test = X_full_test.fillna(' ')

# Predict tập test vs full train, không dùng tập test 

In [8]:
y_pred = clf.predict_proba(X_full_test)[:,1]

dicts = pd.DataFrame({
    'urlid' : df_test['urlid'],
    'label' : y_pred
})

dicts.to_csv('submit.csv', index=False)

NameError: name 'clf' is not defined

# Dùng tập test 

In [9]:
tfidf = TfidfVectorizer(stop_words = stop_words,
                        tokenizer = tokenizer_porter,
                        preprocessor = preprocessor)
tfidf.fit(X)
tfidf.fit(X_full_test)

/home/cuong/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(preprocessor=<function preprocessor at 0x7ff615ee4b70>,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function tokenizer_porter at 0x7ff615ee4950>)

In [10]:
X_tfidf = tfidf.transform(X)

In [11]:
X_full_test_tfidf = tfidf.transform(X_full_test)

# Train split với cả tập test 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y, test_size=0.2, random_state=102)

In [25]:
clf = Pipeline([('pca',TruncatedSVD(n_components=800)),
                ('clf', VotingClassifier(estimators=[
                    ('clf1',LogisticRegression()),
                    ('clf2',SVC(probability=True)),
                    ('clf3',LGBMClassifier(random_state=42))], voting='soft', weights=[2,2,1]))])
clf.fit(X_train,y_train)

Pipeline(steps=[('pca', TruncatedSVD(n_components=800)),
                ('clf',
                 VotingClassifier(estimators=[('clf1', LogisticRegression()),
                                              ('clf2', SVC(probability=True)),
                                              ('clf3',
                                               LGBMClassifier(random_state=42))],
                                  voting='soft', weights=[2, 2, 1]))])

In [26]:
y_pred = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred)

0.8712850737316402

# Train full tập train với cả tập test 


In [27]:
clf.fit(X_tfidf,Y)

Pipeline(steps=[('pca', TruncatedSVD(n_components=800)),
                ('clf',
                 VotingClassifier(estimators=[('clf1', LogisticRegression()),
                                              ('clf2', SVC(probability=True)),
                                              ('clf3',
                                               LGBMClassifier(random_state=42))],
                                  voting='soft', weights=[2, 2, 1]))])

In [28]:
y_pred = clf.predict_proba(X_full_test_tfidf)[:,1]

dicts = pd.DataFrame({
    'urlid' : df_test['urlid'],
    'label' : y_pred
})

dicts.to_csv('submit.csv', index=False)